# Deploy

In [ ]:
!pip install unsloth transformers fastapi uvicorn pyngrok nest-asyncio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf /content/nutrition_finetuned_model_v2
!cp -r /content/drive/MyDrive/Colab_Models/nutrition_finetuned_model_v2_1/content/nutrition_finetuned_model_v2 /content/nutrition_finetuned_model_v2

In [ ]:
!ls /content/nutrition_finetuned_model_v2

adapter_config.json	   README.md		    tokenizer_config.json
adapter_model.safetensors  runs			    tokenizer.json
checkpoint-60		   special_tokens_map.json


In [ ]:
from fastapi import FastAPI, Request
from unsloth import FastLanguageModel
import torch
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import logging
import json
from fastapi.middleware.cors import CORSMiddleware

#logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

ngrok.set_auth_token(" Set auth token eg. 2vOiEDTwuBQSvknPsjU_2MyJ...")

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/content/nutrition_finetuned_model_v2/",
    max_seq_length=2048,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

@app.post("/chatbot")
async def chatbot(request: Request):
    try:
        body = await request.body()
        body_str = body.decode('utf-8')
        logger.info(f"Raw request body: {body_str}")

        if not body_str:
            return {"error": "Request body is empty"}
        try:
            data = json.loads(body_str)
        except json.JSONDecodeError as e:
            logger.error(f"JSON parsing error: {str(e)}")
            return {"error": f"Invalid JSON: {str(e)}"}

        if "query" not in data:
            return {"error": "Missing 'query' parameter in request"}

        query = data["query"]

        messages = [{"role": "user", "content": query}]
        formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.7, do_sample=True)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return {"response": response}
    except Exception as e:
        logger.error(f"Error processing request: {str(e)}")
        return {"error": f"Server error: {str(e)}"}

@app.get("/")
async def root():
    return {"message": "Nutrition Chatbot API is running! Send POST requests to /chatbot endpoint."}

if __name__ == "__main__":
    nest_asyncio.apply()
    #ngrok tunnel
    public_url = ngrok.connect(8000).public_url
    print(f"Public URL: {public_url}")
    print("""curl -Uri "https://fd82-34-169-57-123.ngrok-free.app/chatbot" -Method POST -ContentType "application/json" -Body '{"query":"give me keto diet advice"}'""")

    uvicorn.run(app, host="0.0.0.0", port=8000)